In [1]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 2: Imports & Configuration
# ═══════════════════════════════════════════════════════════════════════════
import asyncio, re, random
from pathlib import Path
from datetime import datetime
from urllib.parse import urljoin, urlparse
from collections import defaultdict, Counter

import nest_asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, Page, BrowserContext

nest_asyncio.apply()   # Allows asyncio.run() inside Jupyter

# ── Configuration ──────────────────────────────────────────────────────────
BASE_URL    = "https://help.cebupacificair.com"
OUTPUT_FILE = "cebu_pacific_helpcenter.txt"
CONCURRENCY = 4       # Parallel page scrapers (≤5 keeps you polite + hidden)
NAV_TIMEOUT = 60_000  # 60s navigation timeout (ms) — increased for reliability
MIN_DELAY   = 0.5     # Seconds between requests per tab
MAX_DELAY   = 1.1

# Block heavy resources — makes each page load 3-5x faster
BLOCK_TYPES = {"image", "media", "font", "other"}

print("✅ Configuration ready")
print(f"   Base URL    : {BASE_URL}")
print(f"   Output file : {OUTPUT_FILE}")
print(f"   Concurrency : {CONCURRENCY} parallel tabs")


✅ Configuration ready
   Base URL    : https://help.cebupacificair.com
   Output file : cebu_pacific_helpcenter.txt
   Concurrency : 4 parallel tabs


In [2]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 3: Stealth Browser — hides automation signals from the server
# ═══════════════════════════════════════════════════════════════════════════
async def create_stealth_context(playwright):
    """
    Launch headless Chromium with anti-detection patches.
    - Removes 'navigator.webdriver' flag
    - Sets realistic viewport, UA, locale, timezone
    Returns (browser, context).
    """
    browser = await playwright.chromium.launch(
        headless=True,
        args=[
            "--no-sandbox",
            "--disable-blink-features=AutomationControlled",  # Key stealth flag
            "--disable-dev-shm-usage",
            "--disable-extensions",
            "--disable-infobars",
        ],
    )
    context = await browser.new_context(
        viewport={"width": 1366, "height": 768},
        user_agent=(
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/122.0.0.0 Safari/537.36"
        ),
        locale="en-US",
        timezone_id="Asia/Manila",  # Appear as a PH user
        extra_http_headers={
            "Accept-Language": "en-US,en;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
            "Sec-Fetch-User":  "?1",
        },
    )
    # Patch webdriver property on every page inside this context
    await context.add_init_script(
        "Object.defineProperty(navigator, 'webdriver', { get: () => undefined });"
    )
    return browser, context


async def block_heavy_resources(page: Page):
    """Abort image/font/media requests — keeps page loads fast."""
    async def _route(route):
        if route.request.resource_type in BLOCK_TYPES:
            await route.abort()
        else:
            await route.continue_()
    await page.route("**/*", _route)


print("✅ Stealth browser helpers ready")

✅ Stealth browser helpers ready


In [3]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 4: URL Discovery — Homepage → Category Pages → Article URLs
# ═══════════════════════════════════════════════════════════════════════════
async def discover_article_urls() -> dict:
    """
    Two-depth crawl:
      1. Load homepage, collect all internal links
      2. Visit each link, collect article URLs + category name
    Returns: {category_name: [article_url, ...]}
    """
    domain      = urlparse(BASE_URL).netloc
    article_map = defaultdict(set)
    visited     = {BASE_URL}

    async with async_playwright() as pw:
        browser, ctx = await create_stealth_context(pw)
        page = await ctx.new_page()
        await block_heavy_resources(page)

        # ── Level 1: Homepage ──────────────────────────────────────────────
        print(f"📡 Loading homepage...")
        await page.goto(BASE_URL, wait_until="load", timeout=NAV_TIMEOUT)
        await asyncio.sleep(random.uniform(1.2, 1.8))

        home_html = await page.content()
        home_soup = BeautifulSoup(home_html, "lxml")

        lvl1_links = {
            urljoin(BASE_URL, a["href"]).split("#")[0]
            for a in home_soup.find_all("a", href=True)
            if urlparse(urljoin(BASE_URL, a["href"])).netloc == domain
            and urljoin(BASE_URL, a["href"]).split("#")[0] != BASE_URL
        }
        print(f"   Found {len(lvl1_links)} links on homepage")

        # ── Level 2: Category / Section pages ─────────────────────────────
        for i, link in enumerate(lvl1_links):
            print(f"Scanning category pages: {i+1}/{len(lvl1_links)} - {link.split('/')[-1] or 'home'}")
            if link in visited:
                continue
            visited.add(link)

            try:
                await page.goto(link, wait_until="load", timeout=NAV_TIMEOUT)
                await asyncio.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
                html  = await page.content()
                lsoup = BeautifulSoup(html, "lxml")

                # Infer category name from page heading
                h1 = lsoup.find("h1")
                category = (
                    h1.get_text(strip=True) if h1
                    else urlparse(link).path.strip("/")
                                            .replace("-", " ")
                                            .title()
                )

                # Collect all new internal links on this category page
                new_links = {
                    urljoin(BASE_URL, a["href"]).split("#")[0]
                    for a in lsoup.find_all("a", href=True)
                    if urlparse(urljoin(BASE_URL, a["href"])).netloc == domain
                    and urljoin(BASE_URL, a["href"]).split("#")[0] not in visited
                }
                if new_links:
                    article_map[category].update(new_links)

            except Exception as e:
                print(f"   ⚠️  Skipped {link}: {str(e)[:60]}")

        await browser.close()

    result = {k: sorted(v) for k, v in article_map.items() if v}
    total  = sum(len(v) for v in result.values())
    print(f"\n📊 Discovery complete → {total} URLs across {len(result)} categories")
    for cat, urls in sorted(result.items()):
        print(f"   [{len(urls):>3}]  {cat}")
    return result


article_url_map = asyncio.run(discover_article_urls())


📡 Loading homepage...
   Found 29 links on homepage
Scanning category pages: 1/29 - booking-a-flight
Scanning category pages: 2/29 - booking-flights-with-cebu-pacific-121521
Scanning category pages: 3/29 - bali-denpasar-airport-and-travel-guide-189038
Scanning category pages: 4/29 - cancelations-delays
Scanning category pages: 5/29 - ceb-baggage-guidelines-and-upgrade-options-118500
Scanning category pages: 6/29 - cebu-pacific-launches-riyadh-flights-212960
Scanning category pages: 7/29 - cebu-pacific-domestic-travel-guide-162560
Scanning category pages: 8/29 - suspension-of-cebu-pacific-flights-between-cebu-and-222310
Scanning category pages: 9/29 - can-i-bring-this-in-my-hand-carry-123016
Scanning category pages: 10/29 - prepare-for-your-flight
Scanning category pages: 11/29 - can-i-bring-this-in-my-checked-baggage-164532
Scanning category pages: 12/29 - cebu-pacific-enters-damp-lease-agreement-with-bulgaria-206011
Scanning category pages: 13/29 - bangkok-don-mueang-airport-and-trave

In [4]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 5: Article Content Extractor — HTML → Clean Structured Text
# ═══════════════════════════════════════════════════════════════════════════
def extract_article(html: str) -> dict:
    """
    Convert a rendered article page into clean readable text.
    Preserves: headings, paragraphs, bullet lists, numbered lists, tables.
    Removes: nav, footer, ads, scripts, forms.
    """
    soup = BeautifulSoup(html, "lxml")

    # Remove UI noise
    for tag in soup.find_all(["nav", "footer", "header", "aside",
                               "script", "style", "noscript",
                               "form", "button", "svg", "img"]):
        tag.decompose()

    # Extract title
    h1 = soup.find("h1")
    title = h1.get_text(strip=True) if h1 else (
        soup.find("title").get_text(strip=True) if soup.find("title") else "Untitled"
    )

    # Find article body — try common content containers in order of preference
    body_tag = (
        soup.find("article") or
        soup.find(attrs={"class": re.compile(r"article|content|body|post", re.I)}) or
        soup.find("main") or
        soup.body
    )

    lines = []
    seen  = set()  # Deduplicate repeated navigation text

    if body_tag:
        for el in body_tag.descendants:
            if not hasattr(el, "name") or not el.name:
                continue
            text = el.get_text(strip=True)
            if not text or text in seen:
                continue

            if el.name in ("h1", "h2", "h3", "h4"):
                seen.add(text)
                lines.append(f"\n{'#' * int(el.name[1])} {text}")

            elif el.name == "p" and len(text) > 15:
                seen.add(text)
                lines.append(text)

            elif el.name == "li":
                seen.add(text)
                bullet = "- " if el.find_parent("ul") else f"{text[:2]}. ".lstrip()
                lines.append(f"{bullet}{text}")

            elif el.name == "td" and len(text) > 2:
                seen.add(text)
                lines.append(f"| {text}")

    body = re.sub(r"\n{3,}", "\n\n", "\n".join(lines)).strip()
    return {"title": title, "body": body}


print("✅ extract_article() ready")


✅ extract_article() ready


In [5]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 6: Async Batch Scraper — Concurrent Article Fetching
# ═══════════════════════════════════════════════════════════════════════════
async def scrape_one(ctx: BrowserContext, url: str,
                     category: str, sem: asyncio.Semaphore) -> dict | None:
    """Scrape a single article — one tab per article, semaphore-limited."""
    async with sem:
        page = await ctx.new_page()
        await block_heavy_resources(page)
        try:
            await page.goto(url, wait_until="load", timeout=NAV_TIMEOUT)
            await asyncio.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
            content = extract_article(await page.content())
            if not content["body"]:
                return None
            return {
                "url":      url,
                "category": category,
                "title":    content["title"],
                "body":     content["body"],
            }
        except Exception as e:
            print(f"   ❌ {url}: {str(e)[:70]}")
            return None
        finally:
            await page.close()


async def scrape_all(url_map: dict) -> list[dict]:
    """
    Run all article scrapes concurrently under a shared semaphore.
    Uses a single browser + context for connection reuse (fast + stealthy).
    """
    flat = [(cat, url) for cat, urls in url_map.items() for url in urls]
    print(f"🚀 Scraping {len(flat)} articles  (concurrency = {CONCURRENCY})")

    results = []
    sem = asyncio.Semaphore(CONCURRENCY)

    async with async_playwright() as pw:
        browser, ctx = await create_stealth_context(pw)
        coros = [scrape_one(ctx, url, cat, sem) for cat, url in flat]

        for i, fut in enumerate(asyncio.as_completed(coros)):
            print(f"Scraping: {i+1}/{len(coros)}")
            res = await fut
            if res:
                results.append(res)

        await browser.close()

    print(f"\n✅ {len(results)}/{len(flat)} articles scraped successfully")
    return results


scraped_articles = asyncio.run(scrape_all(article_url_map))


🚀 Scraping 238 articles  (concurrency = 4)
Scraping: 1/238
Scraping: 2/238
Scraping: 3/238
Scraping: 4/238
Scraping: 5/238
Scraping: 6/238
Scraping: 7/238
Scraping: 8/238
Scraping: 9/238
Scraping: 10/238
Scraping: 11/238
Scraping: 12/238
Scraping: 13/238
Scraping: 14/238
Scraping: 15/238
Scraping: 16/238
Scraping: 17/238
Scraping: 18/238
Scraping: 19/238
Scraping: 20/238
Scraping: 21/238
Scraping: 22/238
Scraping: 23/238
Scraping: 24/238
Scraping: 25/238
Scraping: 26/238
Scraping: 27/238
Scraping: 28/238
Scraping: 29/238
Scraping: 30/238
Scraping: 31/238
Scraping: 32/238
Scraping: 33/238
Scraping: 34/238
Scraping: 35/238
Scraping: 36/238
Scraping: 37/238
Scraping: 38/238
Scraping: 39/238
Scraping: 40/238
Scraping: 41/238
Scraping: 42/238
Scraping: 43/238
Scraping: 44/238
Scraping: 45/238
Scraping: 46/238
Scraping: 47/238
Scraping: 48/238
Scraping: 49/238
Scraping: 50/238
Scraping: 51/238
Scraping: 52/238
Scraping: 53/238
Scraping: 54/238
Scraping: 55/238
Scraping: 56/238
Scraping: 57/2

In [6]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 7: Save All Content → Single Master TXT File
# ═══════════════════════════════════════════════════════════════════════════
def save_master_txt(articles: list[dict], path: str = OUTPUT_FILE) -> Path:
    """
    Write all scraped articles to one structured txt file.
    Format is designed so you can later split it into JSONL train/val sets.

    Structure per article:
        [INDEX] TITLE
        URL      : ...
        CATEGORY : ...
        ──────────
        [body text]
        ════════════
    """
    out = Path(path)

    header = (
        "=" * 80 + "\n"
        f"SOURCE   : {BASE_URL}\n"
        f"SCRAPED  : {datetime.now().strftime('%Y-%m-%d %H:%M')}\n"
        f"ARTICLES : {len(articles)}\n"
        "=" * 80 + "\n\n"
    )

    by_cat = defaultdict(list)
    for a in articles:
        by_cat[a["category"]].append(a)

    blocks = [header]
    idx    = 0

    for category in sorted(by_cat):
        blocks.append(f"\n{'█' * 80}\n▌ CATEGORY: {category.upper()}\n{'█' * 80}\n")
        for art in by_cat[category]:
            idx += 1
            blocks.append(
                f"\n[{idx:03d}] {art['title']}\n"
                f"URL      : {art['url']}\n"
                f"CATEGORY : {art['category']}\n"
                + "-" * 60 + "\n"
                + art["body"] + "\n"
                + "=" * 80 + "\n"
            )

    out.write_text("".join(blocks), encoding="utf-8")
    kb = out.stat().st_size / 1024
    print(f"✅ Saved → {out.resolve()}")
    print(f"   Size      : {kb:.1f} KB")
    print(f"   Articles  : {len(articles)}")
    print(f"   Categories: {len(by_cat)}")
    return out


output_path = save_master_txt(scraped_articles)


✅ Saved → /Users/twotabs/Documents/GitHub/TwoTabs_James_Dev/DsPy-Learning-Repo/Cebu_Pacific_Ticket_Agent_V2/cebu_pacific_helpcenter.txt
   Size      : 820.3 KB
   Articles  : 199
   Categories: 23


In [7]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 8: Verification — Stats + Sample Preview
# ═══════════════════════════════════════════════════════════════════════════
print("=" * 60)
print("📋 SCRAPE SUMMARY")
print("=" * 60)

cat_counts   = Counter(a["category"] for a in scraped_articles)
body_lengths = [len(a["body"]) for a in scraped_articles]

print(f"\n{'Category':<42} {'Articles':>8}")
print("-" * 52)
for cat, n in sorted(cat_counts.items(), key=lambda x: -x[1]):
    print(f"  {cat:<40} {n:>8}")

print(f"\n📊 Content Stats:")
print(f"   Total articles    : {len(scraped_articles)}")
print(f"   Avg body length   : {sum(body_lengths) // len(body_lengths):,} chars")
print(f"   Min body length   : {min(body_lengths):,} chars")
print(f"   Max body length   : {max(body_lengths):,} chars")

# Sample preview
sample = scraped_articles[0]
print(f"\n📝 Sample Article:")
print(f"   Title    : {sample['title']}")
print(f"   Category : {sample['category']}")
print(f"   URL      : {sample['url']}")
print(f"\n{'─'*60}")
print(sample["body"][:500] + "...")
print(f"\n🎯 Master file ready: {OUTPUT_FILE}")
print("   Next step: Use this file to build accurate JSONL train/val sets!")


📋 SCRAPE SUMMARY

Category                                   Articles
----------------------------------------------------
  Articles/Flying Out Soon                       39
  Booking Flights with Cebu Pacific              28
  Check-in & Boarding                            17
  Rebooking or Canceling Your Flight             16
  Booking a Flight                               15
  Prepare for Your Flight                        10
  Adding or Linking a Booking to Your Account        9
  Cancelations & Delays                           7
  Manage Your Booking                             7
  Hi, how can we help?                            7
  Add-ons & Services                              6
  Baggage & Cargo                                 5
  CEB Baggage Guidelines and Upgrade Options        4
  Suspension of Cebu Pacific Flights Between Cebu and San Vicente        4
  Bangkok (Don Mueang) Airport and Travel Guide        4
  Signing up for a MyCebuPacific Account          4
  Bali (Denp